

> Installing Dependencies




In [ ]:
%pip install numpy pandas matplotlib opencv-python torch shapely supervision ultralytics

In [ ]:
%pip install supervision

**Unzipping Files**

In [ ]:
import zipfile
import os

try:
    # Unzip the image files
    with zipfile.ZipFile(r'C:\Users\ashut\OneDrive\Desktop\ashutosh\ML-OPS\IITG-Sample TAsk\image_chips_native-20250212T103727Z-001.zip', 'r') as zip_ref:
        zip_ref.extractall('images')
    print("Images unzipped successfully!")

    # Unzip the label files
    with zipfile.ZipFile(r'C:\Users\ashut\OneDrive\Desktop\ashutosh\ML-OPS\IITG-Sample TAsk\labels-20250212T103318Z-001.zip', 'r') as zip_ref:
        zip_ref.extractall('labels')
    print("Labels unzipped successfully!")

except zipfile.BadZipFile:
    print("Error: The file is not a valid ZIP file or is corrupted.")
except FileNotFoundError:
    print("Error: The file was not found. Check the file name and path.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

new1

In [ ]:
import glob
import os

In [ ]:
# Define dataset paths
images_path = "images\image_chips_native"
labels_path = "labels\labels\labels_hd"

# List files safely
image_files = glob.glob(os.path.join(images_path, "*.TIF"))
label_files = glob.glob(os.path.join(labels_path, "*.txt"))

print(f"Total images: {len(image_files)}")
print(f"Total label files: {len(label_files)}")

shot2

In [ ]:
import json
import numpy as np

solar_panel_count = 0
label_counts = []

for file in label_files:
    with open(file, 'r') as f:
        lines = f.readlines()
        label_counts.append(len(lines))
        solar_panel_count += len(lines)

print("Total instances of solar panels:", solar_panel_count)
print("Label distribution:", np.bincount(label_counts))

Area and other stats

In [ ]:
import matplotlib.pyplot as plt

areas = []

for file in label_files:
    with open(file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            values = line.strip().split()
            if len(values) == 5:
                _, _, _, width, height = map(float, values)
                areas.append(width * height)

print("Mean Area:", np.mean(areas))
print("Standard Deviation:", np.std(areas))

plt.hist(areas, bins=20, edgecolor='black')
plt.xlabel('Area (sq meters)')
plt.ylabel('Frequency')
plt.title('Distribution of Solar Panel Areas')
plt.show()

Computing IoU using Shapely

In [ ]:
from shapely.geometry import box

def yolo_to_shapely_box(yolo_box):
    x_c, y_c, w, h = yolo_box
    x_min = x_c - (w / 2)
    y_min = y_c - (h / 2)
    x_max = x_c + (w / 2)
    y_max = y_c + (h / 2)
    return box(x_min, y_min, x_max, y_max)

def calculate_iou(box1, box2):
    intersection = box1.intersection(box2).area
    union = box1.union(box2).area
    return intersection / union if union > 0 else 0

In [ ]:
from sklearn.metrics import precision_recall_curve, auc

def compute_ap(y_true, y_scores):
    precision, recall, _ = precision_recall_curve(y_true, y_scores)
    return auc(recall, precision)

YOLO Model

In [ ]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [ ]:
import yaml
with open("solar_dataset.yaml", 'r') as f:
    data = yaml.safe_load(f)
print(f"Validation images directory: {data['val']}")

In [ ]:
import os

# Define dataset directories
dataset_dir = r"C:\Users\ashut\OneDrive\Desktop\ashutosh\ML-OPS\IITG-Sample TAsk"
train_images = os.path.join(dataset_dir, "train/images")
val_images = os.path.join(dataset_dir, "val/images")
test_images = os.path.join(dataset_dir, "test/images")

# Define label directories (assumed to be in the same structure)
train_labels = os.path.join(dataset_dir, "train/labels")
val_labels = os.path.join(dataset_dir, "val/labels")

# Create the YAML file
yaml_content = f"""
path: {dataset_dir}  
train: {train_images}  
val: {val_images}  
test: {test_images}  

nc: 1  
names: ['solar_panel']
"""

yaml_path = os.path.join(dataset_dir, "solar_dataset.yaml")

# Write YAML content to a file
with open(yaml_path, "w") as yaml_file:
    yaml_file.write(yaml_content.strip())

print(f"YAML file created at: {yaml_path}")


In [ ]:
import os
import shutil
import random
from sklearn.model_selection import train_test_split

dataset_root = "C:/Users/ashut/OneDrive/Desktop/ashutosh/ML-OPS/IITG-Sample TAsk"
images_dir = os.path.join(dataset_root, "images\image_chips_native")
labels_dir = os.path.join(dataset_root, "labels\labels\labels_hd")

image_files = [f for f in os.listdir(images_dir) if f.endswith((".jpg", ".JPG", ".jpeg", ".JPEG", ".TIF", ".tif"))] # Handle multiple extensions

train_files, test_files = train_test_split(image_files, test_size=0.2, random_state=42)
train_files, val_files = train_test_split(train_files, test_size=0.1, random_state=42)

def move_files(file_list, src_img_dir, src_label_dir, dest_img_dir, dest_label_dir):
    os.makedirs(dest_img_dir, exist_ok=True)
    os.makedirs(dest_label_dir, exist_ok=True)

    for file_name in file_list:
        shutil.move(os.path.join(src_img_dir, file_name), os.path.join(dest_img_dir, file_name))
        label_name = os.path.splitext(file_name)[0] + ".txt"  # Improved label name handling
        label_path = os.path.join(src_label_dir, label_name)
        if os.path.exists(label_path):
            shutil.move(label_path, os.path.join(dest_label_dir, label_name))
        else:
            print(f"Warning: Label file {label_name} not found for image {file_name}")


move_files(train_files, images_dir, labels_dir, os.path.join(dataset_root, "train", "images"), os.path.join(dataset_root, "train", "labels"))
move_files(val_files, images_dir, labels_dir, os.path.join(dataset_root, "val", "images"), os.path.join(dataset_root, "val", "labels"))
move_files(test_files, images_dir, labels_dir, os.path.join(dataset_root, "test", "images"), os.path.join(dataset_root, "test", "labels"))

print(f"Train images: {len(train_files)}, Val images: {len(val_files)}, Test images: {len(test_files)}")

In [ ]:
import cv2
import os

# Path to the images folder
image_folder = 'images'

# List all image files
image_files = os.listdir(image_folder)

# Load images into a list
images = []
for image_file in image_files:
    image_path = os.path.join(image_folder, image_file)
    image = cv2.imread(image_path)
    images.append(image)

In [ ]:
import os

# List files in labels_hd
print("Files in labels_hd:")
print(os.listdir('labels\labels\labels_hd'))

# List files in labels_native
print("Files in labels_native:")
print(os.listdir('labels\labels\labels_native'))

In [ ]:
# Example: Inspect the first label file
label_file_path = 'labels\labels\labels_native\solarpanels_native_1__x0_0_y0_6845_dxdy_416.txt'  # Replace with your file path
with open(label_file_path, 'r') as f:
    lines = f.readlines()
    print("Contents of the label file:")
    for line in lines:
        print(line.strip())

**Load YOLO Labels:**

In [ ]:
def load_yolo_labels(label_file_path):
    with open(label_file_path, 'r') as f:
        lines = f.readlines()

    labels = []
    for line in lines:
        class_id, x_center, y_center, width, height = map(float, line.strip().split())
        labels.append({
            'class_id': int(class_id),
            'x_center': x_center,
            'y_center': y_center,
            'width': width,
            'height': height
        })
    return labels

# Example usage
label_file_path = 'labels\labels\labels_native\solarpanels_native_1__x0_0_y0_6845_dxdy_416.txt'  # Replace with your file path
labels = load_yolo_labels(label_file_path)
print("Loaded labels:", labels)

**Code to Match Images with Labels:**

In [ ]:
import os

# Example: Match an image with its label file
image_folder = 'images/image_chips_native'
label_folder = 'labels/labels/labels_native'

# List all image files
image_files = os.listdir(image_folder)

for image_file in image_files:
    # Get the corresponding label file
    label_file = os.path.splitext(image_file)[0] + '.txt'
    label_file_path = os.path.join(label_folder, label_file)

    if os.path.exists(label_file_path):
        print(f"Image: {image_file} -> Label: {label_file}")
        # Load the labels for this image
        labels = load_yolo_labels(label_file_path)
        print("Labels for this image:", labels)
    else:
        print(f"Label file not found for image: {image_file}")

In [ ]:
%pip install opencv-python

import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Load the image
image_path = os.path.join(image_folder, image_file)
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB for matplotlib

# Plot the image
fig, ax = plt.subplots(1)
ax.imshow(image)

# Draw bounding boxes for the labels
for label in labels:
    # Convert YOLO format to bounding box coordinates
    x_center = label['x_center'] * image.shape[1]
    y_center = label['y_center'] * image.shape[0]
    width = label['width'] * image.shape[1]
    height = label['height'] * image.shape[0]
    x_min = x_center - (width / 2)
    y_min = y_center - (height / 2)

    # Draw the bounding box
    rect = patches.Rectangle((x_min, y_min), width, height, linewidth=2, edgecolor='r', facecolor='none')
    ax.add_patch(rect)

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Example: Compute areas (assuming width and height are in meters)
areas = []
for label_file in os.listdir(label_folder):
    label_file_path = os.path.join(label_folder, label_file)
    with open(label_file_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            _, _, _, width, height = map(float, line.strip().split())
            area = width * height  # Assuming width and height are in meters
            areas.append(area)

# Calculate mean and standard deviation
mean_area = np.mean(areas)
std_area = np.std(areas)
print(f"Mean area: {mean_area:.2f} m²")
print(f"Standard deviation: {std_area:.2f} m²")

# Plot histogram
plt.hist(areas, bins=20, edgecolor='black')
plt.xlabel('Area (m²)')
plt.ylabel('Frequency')
plt.title('Distribution of Solar Panel Areas')
plt.show()

In [ ]:
from shapely.geometry import box

def calculate_iou(box1, box2):
    """
    Calculate IoU between two bounding boxes using Shapely.
    Bounding boxes are in Ultralytics YOLO format: (x_center, y_center, width, height).
    """
    # Convert YOLO format to (x_min, y_min, x_max, y_max)
    def yolo_to_corners(yolo_box):
        x_center, y_center, width, height = yolo_box
        x_min = x_center - (width / 2)
        y_min = y_center - (height / 2)
        x_max = x_center + (width / 2)
        y_max = y_center + (height / 2)
        return (x_min, y_min, x_max, y_max)

    # Convert boxes to Shapely polygons
    box1_poly = box(*yolo_to_corners(box1))
    box2_poly = box(*yolo_to_corners(box2))

    # Compute intersection and union
    intersection = box1_poly.intersection(box2_poly).area
    union = box1_poly.union(box2_poly).area

    # Compute IoU
    iou = intersection / union if union > 0 else 0
    return iou
    print(iou)

In [ ]:
# Example boxes in YOLO format
#box1_poly = (0.5, 0.5, 0.4, 0.4)
#box2_poly = (0.6, 0.6, 0.4, 0.4)

# Compute IoU using Shapely
iou_shapely = calculate_iou(box1_poly, box2_poly)

print(f"IoU (Shapely): {iou_shapely:.4f}")

In [ ]:
%pip install torchvision

In [ ]:
%pip install supervision

In [ ]:
# Function to load YOLO labels
def load_yolo_labels(label_file_path):
    with open(label_file_path, 'r') as f:
        lines = f.readlines()

    labels = []
    for line in lines:
        class_id, x_center, y_center, width, height = map(float, line.strip().split())
        labels.append({
            'class_id': int(class_id),
            'x_center': x_center,
            'y_center': y_center,
            'width': width,
            'height': height
        })
    return labels

# Function to convert YOLO format to absolute coordinates
def yolo_to_absolute(label, image_width, image_height):
    x_center = label['x_center'] * image_width
    y_center = label['y_center'] * image_height
    width = label['width'] * image_width
    height = label['height'] * image_height
    x_min = x_center - (width / 2)
    y_min = y_center - (height / 2)
    x_max = x_center + (width / 2)
    y_max = y_center + (height / 2)
    return x_min, y_min, x_max, y_max

# Example usage
image_folder = 'images/image_chips_native'
label_folder = 'labels/labels/labels_native'

# List all image files
image_files = os.listdir(image_folder)

for image_file in image_files:
    # Get the corresponding label file
    label_file = os.path.splitext(image_file)[0] + '.txt'
    label_file_path = os.path.join(label_folder, label_file)

    if os.path.exists(label_file_path):
        print(f"Image: {image_file} -> Label: {label_file}")
        # Load the labels for this image
        labels = load_yolo_labels(label_file_path)
        print("Labels for this image:", labels)

        # Load the image
        image_path = os.path.join(image_folder, image_file)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Plot the image with bounding boxes
        fig, ax = plt.subplots(1)
        ax.imshow(image)

        for label in labels:
            x_min, y_min, x_max, y_max = yolo_to_absolute(label, image.shape[1], image.shape[0])
            width = x_max - x_min
            height = y_max - y_min
            rect = patches.Rectangle((x_min, y_min), width, height, linewidth=2, edgecolor='r', facecolor='none')
            ax.add_patch(rect)

        plt.show()
    else:
        print(f"Label file not found for image: {image_file}")